In [1]:
import pandas as pd
import sys

In [ ]:
# Define input and output filenames
sentiment_file = 'SENT.csv'
brent_file = 'Final brent_processed_data.csv' # Assuming this file exists with Date, Close, Volume columns
USDX_file = 'Final USDX_processed_data'
output_file = 'SENT_process.csv'

try:
    print(f"Reading {sentiment_file}...")
    sent_df = pd.read_csv(sentiment_file)
    print(f"Read {len(sent_df)} rows from {sentiment_file}.")
    print(f"Columns: {sent_df.columns.tolist()}")

    print(f"Reading {brent_file}...")
    brent_df = pd.read_csv(brent_file)
    print(f"Read {len(brent_df)} rows from {brent_file}.")
    print(f"Columns: {brent_df.columns.tolist()}")

    # --- Data Cleaning and Preparation ---
    # Ensure required columns exist
    required_sent_cols = ['Date','Daily_Sum_RavenPack','Csum_RavenPack','Daily_Sum_FinBERT','Csum_FinBERT','Daily_Sum_CrudeBERT','Csum_CrudeBERT','Daily_Sum_CrudeBERT_GT','Csum_CrudeBERT_GT','Daily_Sum_CrudeBERT_Plus','Csum_CrudeBERT_Plus','Daily_Sum_CrudeBERT_Plus_GT','Csum_CrudeBERT_Plus_GT']
    required_brent_cols = ['Date', 'Close', 'Volume']
    if not all(col in sent_df.columns for col in required_sent_cols):
        print(f"Error: Missing required columns in {sentiment_file}. Expected: {required_sent_cols}")
        sys.exit(1)
    if not all(col in brent_df.columns for col in required_brent_cols):
        print(f"Error: Missing required columns in {brent_file}. Expected: {required_brent_cols}")
        sys.exit(1)

    # Convert 'Date' columns to datetime objects
    # Using errors='coerce' will turn unparseable dates into NaT (Not a Time)
    print("Converting 'Date' columns to datetime objects...")
    sent_df['Date'] = pd.to_datetime(sent_df['Date'], errors='coerce')
    brent_df['Date'] = pd.to_datetime(brent_df['Date'], errors='coerce')

    # Normalize dates to remove time component before merging
    print("Normalizing dates (removing time component)...")
    sent_df['Date'] = sent_df['Date'].dt.normalize()
    brent_df['Date'] = brent_df['Date'].dt.normalize()

    # Drop rows where date conversion failed
    sent_rows_before = len(sent_df)
    brent_rows_before = len(brent_df)
    sent_df.dropna(subset=['Date'], inplace=True)
    brent_df.dropna(subset=['Date'], inplace=True)
    if len(sent_df) < sent_rows_before:
        print(f"Warning: Dropped {sent_rows_before - len(sent_df)} rows from {sentiment_file} due to invalid dates.")
    if len(brent_df) < brent_rows_before:
        print(f"Warning: Dropped {brent_rows_before - len(brent_df)} rows from {brent_file} due to invalid dates.")

    # Ensure 'Close' and 'Volume' are numeric, coercing errors
    print("Converting 'Close' and 'Volume' to numeric...")
    brent_df['Close'] = pd.to_numeric(brent_df['Close'], errors='coerce')
    brent_df['Volume'] = pd.to_numeric(brent_df['Volume'], errors='coerce')
    brent_df.dropna(subset=['Close', 'Volume'], inplace=True) # Drop rows where conversion failed

    # --- Merge DataFrames ---
    print("Merging dataframes on 'Date'...")
    # Using an inner merge keeps only dates present in both files
    merged_df = pd.merge(sent_df, brent_df, on='Date', how='inner')
    print(f"Merged dataframe has {len(merged_df)} rows.")

    if len(merged_df) == 0:
        print("Error: No matching dates found between the two files. Cannot proceed.")
        sys.exit(1)

    # Sort by date to ensure correct difference calculation
    print("Sorting merged data by date...")
    merged_df.sort_values(by='Date', inplace=True)

    # --- Calculate Differences ---
    print("Calculating price and volume differences...")
    # .diff() calculates the difference between the current row and the previous row
    merged_df['Price_Difference'] = merged_df['Close'].diff()
    merged_df['Volume_Difference'] = merged_df['Volume'].diff()

    # The first row will have NaN for differences, which is expected

    # --- Select and Order Columns ---
    output_columns = ['Date', 'BRENT Close', 'BRENT Volume', 'USDX Close','USDX Volume','Price_Difference', 'Volume_Difference','Daily_Sum_RavenPack','Csum_RavenPack','Daily_Sum_FinBERT','Csum_FinBERT','Daily_Sum_CrudeBERT','Csum_CrudeBERT','Daily_Sum_CrudeBERT_GT','Csum_CrudeBERT_GT','Daily_Sum_CrudeBERT_Plus','Csum_CrudeBERT_Plus','Daily_Sum_CrudeBERT_Plus_GT','Csum_CrudeBERT_Plus_GT']
    final_df = merged_df[output_columns]

    # --- Save Output ---
    print(f"Saving results to {output_file}...")
    final_df.to_csv(output_file, index=False, date_format='%Y-%m-%d %H:%M:%S') # Keep original date format if possible

    print("Processing complete.")

except FileNotFoundError as e:
    print(f"Error: File not found - {e}. Please ensure both '{sentiment_file}' and '{brent_file}' exist.")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)


Reading SENT.csv...
Read 2306 rows from SENT.csv.
Columns: ['Date', 'Daily_Sum_RavenPack', 'Csum_RavenPack', 'Daily_Sum_FinBERT', 'Csum_FinBERT', 'Daily_Sum_CrudeBERT', 'Csum_CrudeBERT', 'Daily_Sum_CrudeBERT_GT', 'Csum_CrudeBERT_GT', 'Daily_Sum_CrudeBERT_Plus', 'Csum_CrudeBERT_Plus', 'Daily_Sum_CrudeBERT_Plus_GT', 'Csum_CrudeBERT_Plus_GT']
Reading brent_processed_data.csv...
Read 3740 rows from brent_processed_data.csv.
Columns: ['Date', 'Close', 'Volume']
Converting 'Date' columns to datetime objects...
Normalizing dates (removing time component)...
Converting 'Close' and 'Volume' to numeric...
Merging dataframes on 'Date'...
Merged dataframe has 2306 rows.
Sorting merged data by date...
Calculating price and volume differences...
Saving results to SENT_process.csv...
Processing complete.
